In [ ]:
using FiniteDifferenceMethods

# Mesh

In [ ]:
const T = Float64

n = 5 # number of unknowns (2 ^ p + 1 for exact arithmetics)
o = 1  # width of ghost layer

inds = range(1+o, n+o)

@assert o ≥ 1

In [ ]:
x = collocated(n+2o, T, start=first(inds), stop=last(inds))
xm = staggered(n+2o, T, start=first(inds), stop=last(inds))

h = step(x)

@assert isequal(step(x), step(xm))

# Apertures

Let $a \equiv 0$, $b = 1$ and $s \in \left ( a, b \right )$ (the interface position). Assume fluid domain occupies
$$
\Omega \equiv \left ( s, b \right ).
$$

In [ ]:
s = x[o+1] + h / 4 # interface position

@assert x[o+1] < s < xm[o+1] < x[o+2]

In [ ]:
# moving boundary

A = [T(el > s) for el in x]
B = [T(el > s) for el in xm]
W = [max(min(el+h/2-s, h), 0) for el in x];

In [ ]:
# stationnary boundary

inds = range(1+o, n+o)
A[last(inds):end] .= 0
B[last(inds):end] .= 0
W[last(inds)+1:end] .= 0

W[last(inds)] = h / 2;

# Operators

In [ ]:
row = col = CartesianIndices((inds,))

Gω = Gradient{:x,:ω}((A, B, W), row, col)
Gγ = Gradient{:x,:γ}((A, B, W), row, col);